In [1]:
# ffmpeg command to conver video to images:
#  ffmpeg -i escalator.mp4 -vf fps=1,scale=120:-1 out%d.bmp

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc
import numpy as np
from glob import glob

In [23]:
def rgb2gray(rgb):
    r, g, b = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray / 255.

fdir = 'videos/crow_images'
names = sorted(glob(fdir + "/*.bmp"))
d1, d2, channels = misc.imread(names[0]).shape
d1 = 128
d2 = 160
num = len(names)
X = np.zeros((d1, d2, num))
for n, i in enumerate(names):
    X[:, :, n] = misc.imresize(rgb2gray(misc.imread(i).astype(np.double)) / 255., (d1, d2))
    
X = X.reshape(d1 * d2, num)
print(X.shape)
print(d1)
print(d2)

(20480L, 19L)
128
160


In [7]:
import numpy as np
from numpy.linalg import norm, svd
from scipy.io import loadmat, savemat

def inexact_augmented_lagrange_multiplier(X, lmbda=.01, tol=1e-3,
                                          maxiter=100, verbose=True):
    """
    Inexact Augmented Lagrange Multiplier
    """
    Y = X
    norm_two = norm(Y.ravel(), 2)
    norm_inf = norm(Y.ravel(), np.inf) / lmbda
    dual_norm = np.max([norm_two, norm_inf])
    Y = Y / dual_norm
    A = np.zeros(Y.shape)
    E = np.zeros(Y.shape)
    dnorm = norm(X, 'fro')
    mu = 1.25 / norm_two
    rho = 1.5
    sv = 10.
    n = Y.shape[0]
    itr = 0
    while True:
        Eraw = X - A + (1 / mu) * Y
        Eupdate = np.maximum(Eraw - lmbda / mu, 0) + np.minimum(Eraw + lmbda / mu, 0)
        U, S, V = svd(X - Eupdate + (1 / mu) * Y, full_matrices=False)
        svp = (S > 1 / mu).shape[0]
        if svp < sv:
            sv = np.min([svp + 1, n])
        else:
            sv = np.min([svp + round(.05 * n), n])
        Aupdate = np.dot(np.dot(U[:, :svp], np.diag(S[:svp] - 1 / mu)), V[:svp, :])
        A = Aupdate
        E = Eupdate
        Z = X - A - E
        Y = Y + mu * Z
        mu = np.min([mu * rho, mu * 1e7])
        itr += 1
        if ((norm(Z, 'fro') / dnorm) < tol) or (itr >= maxiter):
            break
    if verbose:
        print("Finished at iteration %d" % (itr))  
    return A, E

In [25]:
numFrames = 19
A, E = inexact_augmented_lagrange_multiplier(X)
A = A.reshape(d1, d2, numFrames) * 255.
E = E.reshape(d1, d2, numFrames) * 255.
#Refer to them by position desired for video demo later 
savemat("./IALM_background_subtraction.mat", {"1": A, "2": E})
print("RPCA complete")

Finished at iteration 15
RPCA complete


In [26]:
import os
import sys
import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
from matplotlib import cm
import matplotlib

#demo inspired by / stolen from @kuantkid on Github - nice work!
def mlabdefaults():
    matplotlib.rcParams['lines.linewidth'] = 1.5
    matplotlib.rcParams['savefig.dpi'] = 300 
    matplotlib.rcParams['font.size'] = 22
    matplotlib.rcParams['font.family'] = "Times New Roman"
    matplotlib.rcParams['legend.fontsize'] = "small"
    matplotlib.rcParams['legend.fancybox'] = True
    matplotlib.rcParams['lines.markersize'] = 10
    matplotlib.rcParams['figure.figsize'] = 8, 5.6
    matplotlib.rcParams['legend.labelspacing'] = 0.1
    matplotlib.rcParams['legend.borderpad'] = 0.1
    matplotlib.rcParams['legend.borderaxespad'] = 0.2
    matplotlib.rcParams['font.monospace'] = "Courier New"
    matplotlib.rcParams['savefig.dpi'] = 200
    
def make_video(alg, cache_path='videos/tmp'):
    name = alg
    if not os.path.exists(cache_path):
        os.mkdir(cache_path)

    if not os.path.exists('%s/%s_tmp'%(cache_path, name)):
        os.mkdir("%s/%s_tmp"%(cache_path, name))
    mat = loadmat('./%s_background_subtraction.mat'%(name))
    org = X.reshape(d1, d2, X.shape[1]) * 255.
    fig = plt.figure()
    ax = fig.add_subplot(111)
    usable = [x for x in sorted(mat.keys()) if "_" not in x][0]
    sz = min(org.shape[2], mat[usable].shape[2])
    for i in range(sz):
        ax.cla()
        ax.axis("off")
        ax.imshow(np.hstack([mat[x][:, :, i] for x in sorted(mat.keys()) if "_" not in x] + \
                            [org[:, :, i]]), cm.gray)
        fname_ = '%s/%s_tmp/_tmp%03d.png'%(cache_path, name, i)
        if (i % 25) == 0:
            print('Completed frame', i, 'of', sz, 'for method', name)
        fig.tight_layout()
        fig.savefig(fname_, bbox_inches="tight")
        #Write out an mp4 and webm video from the png files. -r 5 means 5 frames a second
        #libx264 is h.264 encoding, -s 160x130 is the image size
        #You may need to sudo apt-get install libavcodec
    plt.close()

    num_arrays = na = len([x for x  in mat.keys() if "_" not in x])
    cdims = (na * d1, d2)
    # comment out since code below isn't producing video
    # run from cmd line instead: $ ffmpeg -y -r 10 -i '_tmp%03d.png' -c:v libx264 -s 256x160 -preset ultrafast -pix_fmt yuv420p animation.mp4
#     print cdims
#     cmd_h264 = "ffmpeg -y -r 10 -i '%s/%s_tmp/_tmp%%03d.png' -c:v libx264 " % (cache_path, name) + \
#                "-s %dx%d -preset ultrafast -pix_fmt yuv420p ~/%s_animation.mp4" % (cdims[0], cdims[1],name)
#     cmd_vp8 = "ffmpeg -y -r 10 -i '%s/%s_tmp/_tmp%%03d.png' -c:v libvpx " % (cache_path, name) + \
#               "-s %dx%d -preset ultrafast -pix_fmt yuv420p ~/%s_animation.webm" % (cdims[0], cdims[1], name)
#     os.system(cmd_h264)
#     os.system(cmd_vp8)
#     print 'made video'
        
if __name__ == "__main__":
    mlabdefaults()
    all_methods = ['IALM']
    for name in all_methods:
        make_video(name);

('Completed frame', 0, 'of', 19L, 'for method', 'IALM')


In [21]:
from IPython.display import HTML
from base64 import b64encode

def html5_video(alg, frames, path):
    #This *should* support all browsers...
    framesz = 21
    info = {"mp4": {"ext":"mp4", "encoded": '', "size":(frames * framesz, framesz)}}
    html_output = []
    for k in info.keys():
        f = open(path, "rb").read()
        encoded = b64encode(f).decode('ascii')
        video_tag = '<video width="500" height="250" autoplay="autoplay" ' + \
                    'loop src="data:video/%s;base64,%s">' % (k, encoded)
        html_output.append(video_tag)
    return HTML(data=''.join(html_output))

In [22]:
html5_video('IALM', 3, 'videos/processed/animation.mp4')

In [27]:
html5_video('IALM', 3, 'videos/processed/crow/animation.mp4')